In [1]:
import math
from tensorboardX import SummaryWriter
import roboschool
sess = None

In [2]:
import tensorflow as tf
import collections
gpu_options = tf.GPUOptions(allow_growth=True,per_process_gpu_memory_fraction=0.8)
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
import gym
import wrappers
env_name = 'SuperMarioBros-v1'
#env = wrappers.make_atari_deepmind(env_name, skip=4)
#env_name = 'RoboschoolAnt-v1'#'CartPole-v1' #'CarRacing-v0' #'LunarLander-v2' #'Acrobot-v1' #
#env = gym.make(env_name)
from a2c import create_super_mario_env

env = create_super_mario_env()

observation_shape = env.observation_space.shape
n_actions = env.action_space.n
shape =(None, ) + observation_shape
print(shape)
print(n_actions)

/home/trrrrr/anaconda3/envs/rl/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


(None, 84, 84, 4)
7


In [4]:
# The typical imports
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Imports specifically so we can render outputs in Jupyter.
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    #plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi = 72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))

In [ ]:
class EnvCreator(object):
    def __init__(self, name):
        self.name = name
    def __call__(self, ):
        return gym.make(self.name)

In [ ]:
from a2c import A2CAgent
import tr_helpers
import networks
import ray
ray.init()

a2c_config = {
    'GAMMA' : 0.99,
    'TAU' : 0.9,
    'LEARNING_RATE' : 1e-3,
    'NAME' : 'Supermario',
    'SCORE_TO_WIN' : 100500,
    'EPISODES_TO_LOG' : 20, 
    'LIVES_REWARD' : 5,
    'GRAD_NORM' : 0.5,
    'ENTROPY_COEF' : 0.001,
    'NUM_ACTORS' : 1,
    'STEPS_NUM' : 50,
    'TRUNCATE_GRADS' : True,
    'ENV_NAME' : env_name,
    'PPO' : False,
    'E_CLIP' : 0.2
}
#self, sess, env_name, observation_shape, actions_num, config = default_config
observation_shape = env.observation_space.shape
actions_num = env.action_space.n
action_shape = env.action_space.shape
print(action_shape)
agent = A2CAgent(sess,'pong', observation_shape, actions_num, action_shape, a2c_config)
agent.train()

2019-04-28 20:50:45,596	INFO node.py:278 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-28_20-50-45_26084/logs.
2019-04-28 20:50:45,719	INFO services.py:396 -- Waiting for redis server at 127.0.0.1:61811 to respond...
2019-04-28 20:50:45,872	INFO services.py:396 -- Waiting for redis server at 127.0.0.1:10056 to respond...
2019-04-28 20:50:45,882	INFO services.py:798 -- Starting Redis shard with 10.0 GB max memory.
2019-04-28 20:50:45,954	INFO services.py:1360 -- Starting the Plasma object store with 13.477717606 GB memory using /dev/shm.



View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=d618bee6883cebda8c691ef9c944b6f3d6f0e958816aa613

()
Frames per seconds:  128.6718359913155
Frames per seconds:  241.3590348455984
Frames per seconds:  261.2203249310438
saving next best rewards:  953.0


In [ ]:
def evaluate(env,t_max=10000):
    rewards = []
    env._max_episode_steps = 9999
    print('reset')
    #env = env.old_env
    s = env.reset()
    reward = 0
    for it in range(t_max):
        action = agent.get_action(s, True)
        ad = agent.get_action_distribution([s])
        #print(ad)
        s, r, done, _ = env.step(action)
        reward += r
        
            
        if done:
            break       
        
    return reward

import gym.wrappers

#env_monitor = wrappers.make_atari_deepmind(env_name, noop_max=30, skip=4)
#env_monitor = wrappers.ReallyDoneWrapper(env_monitor)
env_monitor = gym.wrappers.Monitor(env,directory='video_ppo',force=True)

sessions = [print('reward:', evaluate(env_monitor)) for _ in range(1)]
env_monitor.close()

In [ ]:
gym.envs.registry.all()